In [ ]:
#This Deonstrates the Workflow with the API from API.py

import API
import torch
import numpy as np

In [ ]:
definition_model1 = {
    "processor": "Processor_cond",
    "cond_inds": torch.tensor([10,11,12,13]),
    "processor_args": {"N_min": 500, "percentile2": 95},
    "processor_data": {"folder": "all_sims"},
    "flow_hyper": {"n_layers":14, "dim_notcond": 10, "dim_cond": 4, "CL":"NSF_CL2", "K": 10, "B":3, "network":"MLP", "network_args":torch.tensor([128,4,0.2])},
    "subset_params": {"cond_fn": "cond_M_stars_2age_avZ", "use_fn": "all_galaxies", "train_use_fn_constructor": "construct_all_galaxies_leavout", "leavout_indices": None},
    "data_prep_args": {"transformation_functions":("np.log10",), "transformation_indices":(torch.tensor([10]),), "inverse_transformations":("10**x",)}
} #Example definition like in API.py

In [ ]:
model1 = API.GalacticFlow(definition_model1)

In [ ]:
model1.prepare()

In [ ]:
#Train model.
#Often we want to train in a dedicated .py script that is started with e.g. nohup because training can take a long time.
model1.train(1,0.0004, 1024, 0.998)

In [ ]:
#For this reson we wrote cond_trainer.py, which can be run in a separate process, and will save the model to a file.
#We export the data to files:
model1.cond_trainer_export(1,0.0002, 1024, 0.998, "GF_123456")
#And then run cond_trainer.py in a separate process e.g. with:
#nohup python3 cond_trainer.py &

In [ ]:
#Andthen we collect the flow back again:
#The output of the script is automatically deleted.
model1.cond_trainer_import("GF_123456")

In [ ]:
data_galaxy = model1.Data_sub_v[5]

model_galaxy = model1.sample_galaxy(len(data_galaxy), data_galaxy[0][-4:])

In [ ]:
import res_flow_vis as visual

In [ ]:
visual.get_result_plots(data_galaxy, model_galaxy, label = "API_TEST")

In [ ]:
model1.cond_inds, model1.processor.mu, model1.processor.std

In [ ]:
model1.save("GF_model1.pth")

In [ ]:
# Somewhere else, or another day/session

model2 = API.GalacticFlow("GF_model1.pth")

In [ ]:
model2.cond_inds, model2.processor.mu, model2.processor.std

In [ ]:
model2.prepare()
data_galaxy = model2.Data_sub_v[5]

model_galaxy = model2.sample_galaxy(len(data_galaxy), data_galaxy[0][-4:], GPUs=[9])

In [ ]:
visual.get_result_plots(data_galaxy, model_galaxy, label = "API_TEST2")